In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import random

In [16]:
# performs k-fold cross-validation of Ridge regression to find optimal value of alpha
# inputs: k - #folds, X, y - dataset, aplhas - list of alphas to choose from
# output: list of mse, for each of possible alphas
def ridge_cv(k, X, y, alphas):
    splits = k_split(k,len(X))
    results = []

    

    return results

# sub-procedure for ridge_cv
# returns k splits as tuples (train_indices, test_indices)
# inputs: k - #folds, l - #rows in the dataset (length)
# output: list of tuples
def k_split(k, l):
    if k > l:
        raise Exception("Wrong k-fold")
    indexes = np.arange(0, l)
    random.shuffle(indexes)
    ratio = 1 / k
    
    train, test = [], []
    for _ in indexes:
        if random.random() < ratio:
            test.append(_)
        else:
            train.append(_)
    splits = [train, test]
    return splits

# load and pre-process the dataset
hitters = pd.read_csv("Hitters.csv").dropna().drop("Player", axis=1)
dummies = pd.get_dummies(hitters[['League', 'Division', 'NewLeague']])
# Drop the column with the independent variable (Salary), and columns for which we created dummy variables
X_ = hitters.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
# Define the feature set X.
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
y = hitters.Salary

alphas = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]

results = ridge_cv(10, X, y, alphas)
# visualize the results
plt.plot(alphas, results, 'ro')
plt.title("MSE for different alpha levels for Ridge Regression")
plt.xlabel('alpha')
plt.ylabel('MSE')
plt.xscale('log')
plt.show()

[[88, 147, 95, 23, 43, 202, 255, 200, 32, 256, 56, 240, 128, 257, 151, 90, 181, 150, 110, 99, 166, 162, 230, 191, 135, 145, 211, 165, 112, 201, 12, 15, 258, 234, 81, 13, 79, 96, 29, 22, 47, 5, 174, 116, 4, 31, 42, 246, 118, 216, 154, 197, 179, 187, 27, 70, 82, 63, 198, 94, 50, 59, 168, 172, 14, 233, 55, 127, 91, 217, 146, 34, 123, 78, 122, 153, 129, 117, 247, 125, 249, 37, 213, 87, 131, 40, 72, 52, 208, 105, 54, 102, 155, 67, 222, 232, 157, 238, 195, 260, 205, 235, 75, 3, 48, 248, 252, 10, 236, 68, 212, 106, 136, 18, 33, 133, 98, 242, 89, 100, 36, 84, 183, 76, 74, 9, 53, 250, 164, 114, 149, 71, 25, 259, 177, 221, 104, 20, 38, 144, 143, 244, 61, 134, 107, 214, 1, 45, 245, 241, 159, 97, 137, 140, 109, 253, 193, 119, 64, 239, 16, 66, 28, 189, 182, 210, 58, 108, 175, 262, 39, 11, 124, 188, 17, 0, 148, 185, 60, 44, 251, 219, 169, 83, 220, 49, 130, 226, 229, 51, 103, 8, 173, 254, 194, 73, 120, 69, 30, 192, 261, 243, 224, 204, 2, 171, 231, 176, 46, 92, 115, 77, 111, 24, 215, 156, 167, 161, 18